In [1]:
import numpy as np
import pandas as pd

In [2]:
kmeans_data = pd.read_csv('kmeans_data.csv')
kmeans_group_num = pd.read_csv('kmeans_group_num.csv')
final_final = pd.read_csv('final_final.csv')#使用者的點擊紀錄資料集(已經BY_user去去除最後兩筆資料)
df_news_all = pd.read_csv('df_news_all.csv')#含新聞內容的資料集，請參考我的目錄
news_info = pd.read_csv('news_info.csv')

In [3]:
user_based_data = pd.read_csv('user_based_data.csv')
user_news_rating = pd.read_csv('user_news_rating.csv')
validation_testing_data = pd.read_csv('validation_testing_data.csv')

# user_based by KNN

In [4]:
from knn_from_scratch import knn, euclidean_distance

def recommend_user(user_query_feature, k_recommendations):
    user_prefer_data = []
#############################################################
    with open('user_based_data.csv', 'r') as md:
##############################################################
        # Discard the first line (headings)
        next(md)    # next() 返回迭代對象的下一個項目

        # Read the data into memory
        for line in md.readlines():
            data_row = line.strip().split(',')  # .strip()去掉 row 頭尾空白 
            user_prefer_data.append(data_row)

    # Prepare the data for use in the knn algorithm by picking
    # the relevant columns and converting the numeric columns
    # to numbers since they were read in as strings
    # 將資料型態轉為 float
    user_recommendation_data = []
    for row in user_prefer_data:
        data_row = list(map(float, row[1:]))
        user_recommendation_data.append(data_row)

    # Use the KNN algorithm to get the 5 user that are most
    # similar to particulr user.
    # recommendation_indices, _ :這樣的寫法是為了只回傳第一個結果(有用的結果)
    
    recommendation_indices, _ = knn(
        user_recommendation_data, user_query_feature, k=k_recommendations,
        distance_fn=euclidean_distance, choice_fn=lambda x: None
    )
    
    user_recommendations = []
    for _, index in recommendation_indices:
        user_recommendations.append(user_prefer_data[index])

    return user_recommendations


In [5]:
#########################################################
###############################################
choice_user_id_index = 70  # index 為 0 的 user
###############################################
#########################################################
main_user_feature = user_based_data.iloc[choice_user_id_index,1:] # feature vector for The Post
recommended_user = recommend_user(user_query_feature=main_user_feature, k_recommendations=1001)

In [6]:
# Print recommended movie titles
recommendation_id_list = []
for recommendation in recommended_user:
    recommendation_id_list.append(recommendation[0])

In [7]:
# 刪除 user 自己的 id
i = -1
for recommendation_user in recommendation_id_list:
    i = i + 1
    if  user_based_data.iloc[choice_user_id_index,0] == recommendation_user:
        recommendation_id_list.pop(i)

In [8]:
#回傳的內容是相似使用者看過目標使用者沒看過的資料
def getNotSeenNewsbyUserid(userid_a,userid_b,csv):
    a_HaveRead=set(csv[csv.user_id==userid_a].news_id)
    b_HaveRead=set(csv[csv.user_id==userid_b].news_id)
    a_NotSeenNews=b_HaveRead-a_HaveRead
    temp_dict={userid_b:list(a_NotSeenNews)}
    return temp_dict

In [9]:
def test(userid,similar_users,pred_time,df,by='user'):
    uid=[]
    nid=[]
    #by使用者數量
    if by=='user':
        for x in similar_users[:TopSimilar_user]:
            temp=getNotSeenNewsbyUserid(userid,x,df)
            uid.extend(temp.keys())
            nid.extend(list(temp.values())[0])

        final_df=df[(df.user_id.isin(uid))&(df.news_id.isin(nid))&(df.time<=int(pred_time))]
    #by新聞數量
    else:
        for x in similar_users:
            if len(nid)<TopNews:
                temp=getNotSeenNewsbyUserid(userid,x,df)
                uid.extend(temp.keys())
                nid.extend(list(temp.values())[0])
            else:
                break

        final_df=df[(df.user_id.isin(uid))&(df.news_id.isin(nid))&(df.time<=int(pred_time))]
        final_df=final_df.head(TopNews)
    return final_df

In [10]:
##################################################################################
#############################################################
#目標user_id
target_user=user_based_data.user_id[choice_user_id_index]
#相似使用者(list)已經依照相似程度大到小排序
similar_user= recommendation_id_list
#兩種方式(預設user)
by='user/news'
#最相似的幾個使用者有看過且目標沒看過的新聞
TopSimilar_user=600
#取出目標沒看過的新聞筆數
TopNews=2000
#要預測的日期(意思是說超過這個時間點發布的新聞不算)
pred_time= validation_testing_data.time.values[validation_testing_data.user_id == target_user][0]
df= user_news_rating
#############################################################
##################################################################################

In [11]:
user_based_recommendation=test(target_user,similar_user,pred_time,df,by='user')

In [12]:
# 有多少人看過該新聞才將新聞納入候選集合
n = 2

In [13]:
news_count = user_based_recommendation.groupby('news_id').size().reset_index(name='counts')
news_count_threshold = news_count[news_count.counts >= n]
news_count_threshold = news_count_threshold.reset_index(drop=True)

In [14]:
user_based_recommendation = user_based_recommendation[user_based_recommendation.news_id.isin(news_count_threshold.news_id)]

In [15]:
user_based_recommendation = user_based_recommendation.reset_index(drop=True)

In [16]:
user_based_recommendation.to_csv('user_based_recommendation.csv',index=0)

# save 要推薦的 user id

In [17]:
# 要推薦的 user_id
recommendation_user_id = user_based_data.user_id[choice_user_id_index]
recommendation_user_id = pd.DataFrame([recommendation_user_id])

In [18]:
recommendation_user_id.to_csv('target_user_id.csv',index=0)

In [19]:
kmeans_data.head()

,user_id,group
0,00039bc1-c70c-404d-9a16-5ec2227061f7,201
1,0007d630-bb89-4682-a59e-fffa85eeee30,50
2,0010e97e-641a-4e39-b811-3b9edcccbc1c,1
3,001129e7-1c73-4cd7-bc6a-a449eed3043a,291
4,0018e637-3354-46d8-9ec8-5886e680e4f4,1


In [20]:
# 該 user 的 group
user_group = kmeans_data[kmeans_data.user_id == recommendation_user_id.iloc[0,:].values[0]]
user_group

,user_id,group
70,00edfca4-4b0a-46fc-820b-1f793debec19,1


In [21]:
# 找出該 group 的人數
kmeans_group_num.group_num[kmeans_group_num.group == int(user_group.group)]

1    8845
Name: group_num, dtype: int64

# 用k-means 的分類結果 train bert

In [22]:
t=['time','user_id','news_id']
final_final = final_final[t]

In [23]:
kmeans_data.head()

,user_id,group
0,00039bc1-c70c-404d-9a16-5ec2227061f7,201
1,0007d630-bb89-4682-a59e-fffa85eeee30,50
2,0010e97e-641a-4e39-b811-3b9edcccbc1c,1
3,001129e7-1c73-4cd7-bc6a-a449eed3043a,291
4,0018e637-3354-46d8-9ec8-5886e680e4f4,1


In [24]:
#找出K群的ID
group = int(user_group.group)
kmeans_data_group = kmeans_data[kmeans_data.group == group]

#找出K群ID中的所有點集資料
kmeans_group_user_list = final_final[final_final.user_id.isin(kmeans_data_group.user_id)]

In [25]:
#取K群所有點集資料的唯一新聞ID
kmeans_group_unique_news=np.unique(kmeans_group_user_list.news_id)

In [26]:
df_news_all.head(3)

,path,date,news_guid,title,news_content,content_remove_html_tag
0,../datasets/news/1999-06-14/0b7a5d15-7bcd-11d3...,19990614074852,0B7A5D15-7BCD-11D3-98FF-00E018A00403,導線架市場重新洗牌新台和順德談策略聯盟 ...,\n\n\n<P>在大廠壓縮小廠生存空間，小廠試圖尋求同業策略聯盟的趨勢下，國內3大導線架廠...,在大廠壓縮小廠生存空間，小廠試圖尋求同業策略聯盟的趨勢下，國內3大導線架廠包括順德、佳茂、旭...
1,../datasets/news/2009-08-06/3b508922-e6a5-4c25...,20090806111315,3B508922-E6A5-4C25-B80F-01E8B36BB835,建築鋼每噸利潤達1000元 中國鐵礦石庫存逼新高 ...,\n<P>精實新聞 2009-08-06 11:13:15 記者 余美慧 報導</P><P>...,精實新聞 2009-08-06 11:13:15 記者 余美慧 報導聯合金屬網統計，截至7月...
2,../datasets/news/2009-08-06/b325cc3e-e7b8-417a...,20090806074535,B325CC3E-E7B8-417A-8177-8BEA7A367F00,昇貿利基產品比重提升、新客戶加入，H2樂觀 ...,\n<P>精實新聞 2009-08-06 07:45:35 記者 楊喻斐 報導</P><P>...,精實新聞 2009-08-06 07:45:35 記者 楊喻斐 報導銲錫製品大廠昇貿(330...


In [27]:
#標記label
# df_news_all['label']=0 # 新增一個  column 為 label，值都是 0
df_news_all['label']=df_news_all['news_guid'].map(lambda x:1 if x in kmeans_group_unique_news else 0)

In [28]:
#查看分布比例
df_news_all.groupby('label').size()

label
0    30875
1    17628
dtype: int64

In [29]:
df_news_all.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag,label
0,../datasets/news/1999-06-14/0b7a5d15-7bcd-11d3...,19990614074852,0B7A5D15-7BCD-11D3-98FF-00E018A00403,導線架市場重新洗牌新台和順德談策略聯盟 ...,\n\n\n<P>在大廠壓縮小廠生存空間，小廠試圖尋求同業策略聯盟的趨勢下，國內3大導線架廠...,在大廠壓縮小廠生存空間，小廠試圖尋求同業策略聯盟的趨勢下，國內3大導線架廠包括順德、佳茂、旭...,0
1,../datasets/news/2009-08-06/3b508922-e6a5-4c25...,20090806111315,3B508922-E6A5-4C25-B80F-01E8B36BB835,建築鋼每噸利潤達1000元 中國鐵礦石庫存逼新高 ...,\n<P>精實新聞 2009-08-06 11:13:15 記者 余美慧 報導</P><P>...,精實新聞 2009-08-06 11:13:15 記者 余美慧 報導聯合金屬網統計，截至7月...,0


In [30]:
# 刪除未發布的時間
li = []
for i in range(len(df_news_all)):
    if df_news_all.date[i] > pred_time:
        li.append(df_news_all.index[i])

In [31]:
df_news_all = df_news_all.drop(li)

In [32]:
# #要丟進BERT的資料集
df_news_all.to_csv('bert_train_data.csv', index = 0)

# 用 knn 找出相似 user 看過， target user 卻沒看過的 news 當 bert 的 test data

In [33]:
user_based_recommendation_unique_news_id=np.unique(user_based_recommendation.news_id)

In [34]:
user_based_recommendation_news = df_news_all[df_news_all.news_guid.isin(user_based_recommendation_unique_news_id)]

In [35]:
user_based_recommendation_news.head(2)

,path,date,news_guid,title,news_content,content_remove_html_tag,label
183,../datasets/news/2018-02-12/9b2e10c5-1eee-4081...,20180212104054,9B2E10C5-1EEE-4081-A3AB-566A99845EF5,《1月營收》出貨迎成長，醫揚/聯合創新高 ...,\n<P>MoneyDJ新聞 2018-02-12 10:40:54 記者 劉莞青 報導</...,MoneyDJ新聞 2018-02-12 10:40:54 記者 劉莞青 報導聯合(4129...,1
184,../datasets/news/2018-02-12/3e3b2e23-2780-4aae...,20180212095805,3E3B2E23-2780-4AAE-9B54-BC01F2E29526,《1月營收》被動元件、擴產各擁題材，多家營收創高 ...,\n<P>MoneyDJ新聞 2018-02-12 09:58:05 記者 林詩茵 報導</...,MoneyDJ新聞 2018-02-12 09:58:05 記者 林詩茵 報導被動元件廠商受...,1


In [36]:
user_based_recommendation_news.groupby('label').size()

label
1    1651
dtype: int64

In [37]:
user_based_recommendation_news.to_csv('bert_test_data.csv', index = 0)